In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

torch.cuda.device_count()

cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')

device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

In [2]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = 'G:/PracaMagisterska/Dane/Poly_Continuous_Audio'
excl_folders = ['MT2']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=3

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, shuffle=True)

# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

print('dataset size: ', len(dataset))
print('train set size: ', len(split.train_sampler))
print('val set size: ', len(split.val_sampler))
print('test set size: ', len(split.test_sampler))
dataset.fx_to_label

dataset size:  130000
train set size:  93600
val set size:  10400
test set size:  26000


{'808': 0,
 'BD2': 1,
 'BMF': 2,
 'DPL': 3,
 'DS1': 4,
 'FFC': 5,
 'MGS': 6,
 'OD1': 7,
 'RAT': 8,
 'RBM': 9,
 'SD1': 10,
 'TS9': 11,
 'VTB': 12}

In [3]:
# Model
setnetcond = models.SettingsNetCondRNN(n_settings=dataset.max_num_settings,
                                       mel_shape=dataset.mel_shape, 
                                       num_embeddings=dataset.num_fx, 
                                       embedding_dim=50,
                                       rnn_hidden_size=128,  # Adjust as needed
                                       rnn_layers=2)          # Adjust as needed

# Optimizer
optimizer = optim.Adam(setnetcond.parameters(), lr=0.001)

# Loss function
loss_func = nn.MSELoss(reduction='mean')

# Print the model architecture
print(setnetcond)

SettingsNetCondRNN(
  (emb): Embedding(13, 50)
  (fc0): Linear(in_features=50, out_features=128, bias=True)
  (lstm): LSTM(88, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=32768, out_features=120, bias=True)
  (batchNorm3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (batchNorm4): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=60, out_features=3, bias=True)
)


In [4]:
# SAVE
models_folder = 'G:/PracaMagisterska/Kod_na_magisterke/gfx-classifier_RNN_version-models_and_results/models'
model_name = '20201022_setnetcond_poly_cont_best'
results_folder = 'G:/PracaMagisterska/Kod_na_magisterke/gfx-classifier_RNN_version-models_and_results/result'
results_subfolder = '20201022_setnetcond_poly_cont'

In [5]:
# TRAIN and TEST SettingsNetCond OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

best_val_correct = 0
early_stop_counter = 0

start = time.time()

for epoch in range(100):
    train_loss, train_correct, train_results = trainer.train_settings_cond_net(
        model=setnetcond,
        optimizer=optimizer, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch,
        loss_function=loss_func, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_settings_cond_net(
        model=setnetcond, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler,
        loss_function=loss_func, 
        device='cpu'
    )
    
    test_loss, test_correct, test_results = trainer.test_settings_cond_net(
        model=setnetcond, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler,
        loss_function=loss_func, 
        device='cpu'
    )
    # save model
    if val_correct > best_val_correct:
        best_val_correct = val_correct
        torch.save(setnetcond, '%s/%s' % (models_folder, model_name))
        early_stop_counter = 0
        print('\n=== saved best model ===\n')
    else:
        early_stop_counter += 1
        
    # append results
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

    if early_stop_counter == 15:
        print('\n--- early stop ---\n')
        break

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[5000/93600 (5%)]	Total Loss: 6.3526	Avg Loss: 0.0013
Train Epoch: 0	[10000/93600 (11%)]	Total Loss: 7.7443	Avg Loss: 0.0008
Train Epoch: 0	[15000/93600 (16%)]	Total Loss: 8.7179	Avg Loss: 0.0006
Train Epoch: 0	[20000/93600 (21%)]	Total Loss: 9.4876	Avg Loss: 0.0005
Train Epoch: 0	[25000/93600 (27%)]	Total Loss: 10.1311	Avg Loss: 0.0004
Train Epoch: 0	[30000/93600 (32%)]	Total Loss: 10.7013	Avg Loss: 0.0004
Train Epoch: 0	[35000/93600 (37%)]	Total Loss: 11.2046	Avg Loss: 0.0003
Train Epoch: 0	[40000/93600 (43%)]	Total Loss: 11.6667	Avg Loss: 0.0003
Train Epoch: 0	[45000/93600 (48%)]	Total Loss: 12.0792	Avg Loss: 0.0003
Train Epoch: 0	[50000/93600 (53%)]	Total Loss: 12.5202	Avg Loss: 0.0003
Train Epoch: 0	[55000/93600 (59%)]	Total Loss: 12.8930	Avg Loss: 0.0002
Train Epoch: 0	[60000/93600 (64%)]	Total Loss: 13.2849	Avg Loss: 0.0002
Train Epoch: 0	[65000/93600 (69%)]	Total Loss: 13.6174	Avg Loss: 0.0002
Train Epoch: 0	[70000/93600 (75%)]	Total Loss: 13.9531	Avg Loss: 0.000

In [6]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  99.55341880341881
Epoch:  45

Accuracy:  99.0673076923077
Epoch:  44

Accuracy:  98.8576923076923
Epoch:  44



In [8]:
import pickle

# Save each variable in a separate pickle file
with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_train_losses'), 'wb') as f:
    pickle.dump(all_train_losses, f)

with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_train_correct'), 'wb') as f:
    pickle.dump(all_train_correct, f)

with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'best_train_results'), 'wb') as f:
    pickle.dump(all_train_results[37], f)

with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_val_losses'), 'wb') as f:
    pickle.dump(all_val_losses, f)

with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_val_correct'), 'wb') as f:
    pickle.dump(all_val_correct, f)

with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'best_val_results'), 'wb') as f:
    pickle.dump(all_val_results[32], f)

with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_test_losses'), 'wb') as f:
    pickle.dump(all_test_losses, f)

with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'all_test_correct'), 'wb') as f:
    pickle.dump(all_test_correct, f)

with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'best_test_results'), 'wb') as f:
    pickle.dump(all_test_results[42], f)

with open('%s/%s/%s.pkl' % (results_folder, results_subfolder, 'fx_labels'), 'wb') as f:
    pickle.dump(list(dataset.fx_to_label.keys()), f)